#### Faiss

Facebook AI Similarity Search (Faiss) is a library for efficient similarity search and clustering of dense vectors. It contains algorithms that search in sets of vectors of any size, up to ones that possibly do not fit in RAM. It also contains supporting code for evaluation and parameter tuning.


In [1]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_ollama import OllamaEmbeddings

loader = TextLoader("speech.txt")
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=30)
docs = text_splitter.split_documents(documents)

In [2]:
docs

[Document(metadata={'source': 'speech.txt'}, page_content='The world must be made safe for democracy. Its peace must be planted upon the tested foundations of political liberty. We have no selfish ends to serve. We desire no conquest, no dominion. We seek no indemnities for ourselves, no material compensation for the sacrifices we shall freely make. We are but one of the champions of the rights of mankind. We shall be satisfied when those rights have been made as secure as the faith and the freedom of nations can make them.\n\nJust because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.\n\nâ€¦'),
 Document(metadata={'source': 'speech.txt'}, page_content='â€¦\n\nIt will be all the easier for us to conduct

In [3]:
embeddings = OllamaEmbeddings(model="mxbai-embed-large")
db = FAISS.from_documents(docs, embeddings)
db

In [4]:
### querying
query = "How does the speaker describe the desired outcome of the war?"
docs = db.similarity_search(query)
docs[0].page_content

'It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed in thus addressing you. There are, it may be, many months of fiery trial and sacrifice ahead of us. It is a fearful thing to lead this great peaceful people into war, into the most terrible and disastrous of all wars, civilization itself seeming to be in the balance. But the right is more precious than peace, and we shall fight for the things which we have always carried nearest our heartsâ€”for democracy, for the right of those who submit to authority to have a voice in their own governments, for the rights and liberties of small nations, for a universal dominion of right by such a concert of free peoples as shall bring peace and safety to all nations and make the world itself at last free.'

#### As a Retriever

We can also convert the vectorstore into a Retriever class. This allows us to easily use it in other LangChain methods, which largely work with retrievers


In [5]:
retriever = db.as_retriever()
docs = retriever.invoke(query)
docs[0].page_content

'It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed in thus addressing you. There are, it may be, many months of fiery trial and sacrifice ahead of us. It is a fearful thing to lead this great peaceful people into war, into the most terrible and disastrous of all wars, civilization itself seeming to be in the balance. But the right is more precious than peace, and we shall fight for the things which we have always carried nearest our heartsâ€”for democracy, for the right of those who submit to authority to have a voice in their own governments, for the rights and liberties of small nations, for a universal dominion of right by such a concert of free peoples as shall bring peace and safety to all nations and make the world itself at last free.'

#### Similarity Search with score

There are some FAISS specific methods. One of them is similarity_search_with_score, which allows you to return not only the documents but also the distance score of the query to them. The returned distance score is L2 distance. Therefore, a lower score is better.


In [6]:
docs_and_score = db.similarity_search_with_score(query)
docs_and_score

[(Document(id='83a48d9b-4b88-4a85-9936-e4e6a226708b', metadata={'source': 'speech.txt'}, page_content='It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed in thus addressing you. There are, it may be, many months of fiery trial and sacrifice ahead of us. It is a fearful thing to lead this great peaceful people into war, into the most terrible and disastrous of all wars, civilization itself seeming to be in the balance. But the right is more precious than peace, and we shall fight for the things which we have always carried nearest our heartsâ€”for democracy, for the right of those who submit to authority to have a voice in their own governments, for the rights and liberties of small nations, for a universal dominion of right by such a concert of free peoples as shall bring peace and safety to all nations and make the world itself at last free.'),
  np.float32(0.75587714)),
 (Document(id='df336cbd-58aa-47b8-a550-e27c3ad285e3', metadata={'source': '

In [7]:
embedding_vector = embeddings.embed_query(query)
embedding_vector

[0.018946256,
 -0.020907978,
 -0.017147576,
 0.027528634,
 -0.043132234,
 -0.0902356,
 -0.036515523,
 0.012488248,
 0.0058142412,
 -0.009280731,
 0.019476706,
 -0.0006871337,
 -0.013419779,
 0.029765721,
 -0.015731994,
 -0.024893124,
 -0.043721776,
 -0.01979966,
 0.004678923,
 0.011088657,
 -0.0045645274,
 -0.026544811,
 -0.024403388,
 0.01950149,
 -0.008665073,
 0.02474997,
 -0.024398578,
 0.019016292,
 0.07097463,
 -0.013774597,
 -0.009743374,
 0.03658505,
 -1.6327926e-05,
 -0.04365519,
 0.02651774,
 -0.036229845,
 0.026295198,
 -0.10844844,
 0.014405718,
 -0.009376364,
 0.025629554,
 0.014706885,
 0.028437505,
 -0.013838218,
 -0.0938002,
 -0.027407782,
 -0.03655967,
 -0.08144077,
 0.026420986,
 -0.014847658,
 -0.022555856,
 -0.0541393,
 0.021441497,
 -0.05258271,
 0.0070632254,
 -0.004465711,
 -0.009198273,
 0.040718637,
 0.015422286,
 0.015565942,
 0.041063465,
 0.014865608,
 -0.013722364,
 -0.042424724,
 0.007760169,
 -0.0035235716,
 0.0022055642,
 0.011870154,
 0.005991825,
 0.01

In [8]:
docs_score = db.similarity_search_by_vector(embedding_vector)
docs_score

[Document(id='83a48d9b-4b88-4a85-9936-e4e6a226708b', metadata={'source': 'speech.txt'}, page_content='It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed in thus addressing you. There are, it may be, many months of fiery trial and sacrifice ahead of us. It is a fearful thing to lead this great peaceful people into war, into the most terrible and disastrous of all wars, civilization itself seeming to be in the balance. But the right is more precious than peace, and we shall fight for the things which we have always carried nearest our heartsâ€”for democracy, for the right of those who submit to authority to have a voice in their own governments, for the rights and liberties of small nations, for a universal dominion of right by such a concert of free peoples as shall bring peace and safety to all nations and make the world itself at last free.'),
 Document(id='df336cbd-58aa-47b8-a550-e27c3ad285e3', metadata={'source': 'speech.txt'}, page_content='â

In [9]:
### Saving And Loading
db.save_local("faiss_index")

In [10]:
new_db = FAISS.load_local(
    "faiss_index", embeddings, allow_dangerous_deserialization=True
)

docs = new_db.similarity_search(query)

In [11]:
docs

[Document(id='83a48d9b-4b88-4a85-9936-e4e6a226708b', metadata={'source': 'speech.txt'}, page_content='It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed in thus addressing you. There are, it may be, many months of fiery trial and sacrifice ahead of us. It is a fearful thing to lead this great peaceful people into war, into the most terrible and disastrous of all wars, civilization itself seeming to be in the balance. But the right is more precious than peace, and we shall fight for the things which we have always carried nearest our heartsâ€”for democracy, for the right of those who submit to authority to have a voice in their own governments, for the rights and liberties of small nations, for a universal dominion of right by such a concert of free peoples as shall bring peace and safety to all nations and make the world itself at last free.'),
 Document(id='df336cbd-58aa-47b8-a550-e27c3ad285e3', metadata={'source': 'speech.txt'}, page_content='â